In [2]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models


# Helper libraries
import numpy as np
import matplotlib.pyplot as plt


#Daten laden
#(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

#Klassen Namen
#class_names = ['0']

#Daten laden und graphisch darstellen

#Datensaetze mit 13000 Datenpunkten (statist. Modell)

# load the mnist training data CSV file into a list
training_data_file = open("TrainCobb-133-100_Regression.txt", 'r') 
training_data_list = training_data_file.readlines()
training_data_file.close()

# load the mnist test data CSV file into a list
test_data_file = open("TestCobb-133-100_Regression.txt", 'r') 
test_data_list = test_data_file.readlines()
test_data_file.close()

#------------------------------------------------------

#Listen fuer Aufnahme aller Trainingsdatensaetze
all_labels_train = []
all_inputs_train = []

# go through all the records in the training data set
for record in training_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = float(all_values[0])
    all_labels_train.append(correct_label)
        
    # scale and shift the inputs
    inputs_train = ((numpy.asfarray(all_values[1:])))
    all_inputs_train.append(inputs_train)
    pass

#Umwandlung der Liste in echtes Array
train_labels = np.array(all_labels_train)       
train_images = np.array(all_inputs_train)  

#77 Bilder mit 133 Zeilen und 100 Spalten
train_images = np.reshape(train_images,(77,133,100))
train_images.shape
#train_labels.shape 

#np.concatenate((train_images,)*3, axis=-1)
#train_images.shape

train_images.shape
#train_labels.shape

#Umstrukturieren des Arrays mit 3 Kanaelen fuer CNN (hier alle Kanaele SW sonst RGB)
train_images_new = np.zeros((77,133,100,3))

for i in range(77):
    for ch in range(3):
        for xx in range(133):
            for yy in range(100):
                train_images_new[i,xx,yy,ch] = train_images[i,xx,yy]
                

#-------------------------------------------------------

#Listen fuer Aufnahme aller Testdatensaetze
all_labels_test = []
all_inputs_test = []

# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = float(all_values[0])
    all_labels_test.append(correct_label)
        
    # scale and shift the inputs
    inputs_test = ((numpy.asfarray(all_values[1:])))
    all_inputs_test.append(inputs_test)
    pass

#----------------------------------------------------------------------------------------

#Umwandeln der Listen in echte Arrays
test_labels = np.array(all_labels_test)       
test_label_1 = test_labels[1]
test_images = np.array(all_inputs_test)


#20 Bilder mit 133 Zeilen und 100 Spalten
test_images = np.reshape(test_images,(20,133,100))
test_images.shape

#test_labels.shape Gibt die Gestalt des Arrays aus
test_images.shape

#np.concatenate((test_images,)*3, axis=-1)

#Neues Array fuer CNN mit 3 Kanaelen (hier sind alle Kanaele gleich SW Bild sonst RGB)
test_images_new = np.zeros((20,133,100,3))

for i in range(20):
    for ch in range(3):
        for xx in range(133):
            for yy in range(100):
                test_images_new[i,xx,yy,ch] = test_images[i,xx,yy]

#Daten normalisieren auf Werte zw. 0-1 (Helligkeiten zw. 0-255)
# Normalize pixel values to be between 0 and 1
#train_images_new, test_images_new = train_images_new / 255.0, test_images_new / 255.0
train_images_new, test_images_new = train_images_new / 50.0, test_images_new / 50.0

#testimage1 = test_images_new[1,0:132,0:99,0:3]
#testimage1 = test_images_new[1]

#-------------------------------------------------------------------------------------

#Daten fuer 10 Bsp. plotten
#plt.figure(figsize=(10,10))
#for i in range(25):
#    plt.subplot(5,5,i+1)
#    plt.xticks([])
#    plt.yticks([])
#    plt.grid(False)
#    plt.imshow(train_images[i], cmap=plt.cm.binary)
#    # The CIFAR labels happen to be arrays, 
#    # which is why you need the extra index
#    plt.xlabel(class_names[train_labels[i]])
#plt.show()

#Schichten des neuronalen Netzes CNN convol. und max-pooling

model = models.Sequential() #Sequentielles Neuronales Netz abwechselnd Convolution und MaxPooling

#Parameter: (1) Anzahl Filter (2) Groesse des Filters (3) Aktiv. Methode (4) Inputform des Arreays
#die restlichen Parameter wie Stride sind standard

model.add(layers.Conv2D(32, (2, 2), activation='relu', input_shape=(133, 100,3)))

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (2, 2), activation='relu'))

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (2, 2), activation='relu'))

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(256, (2, 2), activation='relu'))

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(512, (2, 2), activation='relu'))


#model.summary()

#weitere Schichten des voll vernetzten neuronalen Netzes

model.add(layers.Flatten())  #Array wird umstrukturiert in einen 1 dimensionalen Vektor als Eingabeschicht

model.add(layers.Dense(5500, activation='relu'))  #2. Schicht als Zwischenschicht mit 1000 Knoten

output = model.add(layers.Dense(1, activation='linear'))  #Ausgabeschicht mit 1 Knoten fuer den Cobb Winkel
#outputs = Dense(1, activation='linear')(x)

model.summary()  #Zusammenfassung der Modellparameter

#Erstellung des Netzes mit einem 'Adam' Optimierer und dem Opt. Kriterium SparseCategorialCrossEntropy
#als Opt. Kriterium wird die Genauigkeit verwendet (ergibt sich aus true pos. und true neg.)

model.compile(optimizer='adam',
              #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              loss=tf.keras.losses.MeanAbsoluteError(), #for regression
              metrics=['mse'])  #['accuracy']
              

#Training oder Fitting des Modells mit Trainingsdaten sowie Validierung mit Testdaten, 8 Epochen (Wiederholungen)

history = model.fit(train_images_new, train_labels, epochs=12, 
                    validation_data=(test_images_new, test_labels))

#Test des trainierten Modells an den Testdaten

#test_loss, test_acc = model.evaluate(test_images_new,  test_labels, verbose=2)
#test_loss, test_acc = model.evaluate(test_images_new,  test_labels, verbose=2)
#prediction = model.predict(testimage1, verbose=2)
predictions = model.predict(test_images_new)
score = predictions[2]
score.dtype
score1 = 100*score
print(score1) #
#print(f"Cobb angle is {(100 - score):.2f}%")
#print(score)

#Evaluate model (Darstellung der Ergebnisse)

#plt.plot(history.history['mse'], label='mse')             #Training Accuracy
#plt.plot(history.history['val_mse'], label = 'val_mse')   #Test Accuracy
#plt.xlabel('Epoch')
#plt.ylabel('mse')
#plt.ylim([0, 1])
#plt.legend(loc='lower right')

#print(test_acc)
#print(test_acc)

#test_loss, test_acc = model.evaluate(test_image_1,  test_label_1, verbose=2)
#print(test_acc)

#model prediction for single dataset
predictions = model.predict(test_images_new)

score = predictions[0]
score.dtype
score1 = 100*score
print(score1)

c:\Users\User\anaconda3\envs\Python3_9\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 132, 99, 32)    │           416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 66, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 65, 48, 64)     │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 31, 23, 128)    │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 11, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 14, 10, 256)    │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 5, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 6, 4, 512)      │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12288)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5500)           │    67,589,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         5,501 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 68,292,697 (260.52 MB)

 Trainable params: 68,292,697 (260.52 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 687ms/step - loss: 1.3154 - mse: 4.4137 - val_loss: 0.3102 - val_mse: 0.1202
Epoch 2/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 589ms/step - loss: 0.2899 - mse: 0.1033 - val_loss: 0.1000 - val_mse: 0.0173
Epoch 3/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 570ms/step - loss: 0.1454 - mse: 0.0407 - val_loss: 0.1267 - val_mse: 0.0277
Epoch 4/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 572ms/step - loss: 0.1121 - mse: 0.0218 - val_loss: 0.1065 - val_mse: 0.0212
Epoch 5/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 599ms/step - loss: 0.1091 - mse: 0.0204 - val_loss: 0.0974 - val_mse: 0.0180
Epoch 6/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 575ms/step - loss: 0.0924 - mse: 0.0154 - val_loss: 0.1154 - val_mse: 0.0242
Epoch 7/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 572ms/step - loss: 0.0989 - mse: 0.0185 - val_loss: 0.0977 - val_mse: 0.0189
Epoch 8/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 581ms/step - loss: 0.0956 - mse: 0.0167 - val_loss: 0.1010 - val_mse: 0.0181
Epoch 9/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 583ms/step - loss: 0.0911 - mse: 

In [3]:
predictions = model.predict(test_images_new)

score = predictions[0]
score.dtype
score1 = 100*score
print(score1)


score = predictions[1]
score.dtype
score1 = 100*score
print(score1)

score = predictions[2]
score.dtype
score1 = 100*score
print(score1)

score = predictions[3]
score.dtype
score1 = 100*score
print(score1)

score = predictions[4]
score.dtype
score1 = 100*score
print(score1)

score = predictions[5]
score.dtype
score1 = 100*score
print(score1)

score = predictions[6]
score.dtype
score1 = 100*score
print(score1)

score = predictions[7]
score.dtype
score1 = 100*score
print(score1)

score = predictions[8]
score.dtype
score1 = 100*score
print(score1)

score = predictions[9]
score.dtype
score1 = 100*score
print(score1)

score = predictions[10]
score.dtype
score1 = 100*score
print(score1)

score = predictions[11]
score.dtype
score1 = 100*score
print(score1)

score = predictions[12]
score.dtype
score1 = 100*score
print(score1)

score = predictions[13]
score.dtype
score1 = 100*score
print(score1)

score = predictions[14]
score.dtype
score1 = 100*score
print(score1)


score = predictions[15]
score.dtype
score1 = 100*score
print(score1)

score = predictions[16]
score.dtype
score1 = 100*score
print(score1)

score = predictions[17]
score.dtype
score1 = 100*score
print(score1)

score = predictions[18]
score.dtype
score1 = 100*score
print(score1)

score = predictions[19]
score.dtype
score1 = 100*score
print(score1)

score = predictions[20]
score.dtype
score1 = 100*score
print(score1)


score = predictions[21]
score.dtype
score1 = 100*score
print(score1)

score = predictions[22]
score.dtype
score1 = 100*score
print(score1)

score = predictions[23]
score.dtype
score1 = 100*score
print(score1)

score = predictions[24]
score.dtype
score1 = 100*score
print(score1)

score = predictions[25]
score.dtype
score1 = 100*score
print(score1)

score = predictions[26]
score.dtype
score1 = 100*score
print(score1)

score = predictions[27]
score.dtype
score1 = 100*score
print(score1)

score = predictions[28]
score.dtype
score1 = 100*score
print(score1)

score = predictions[29]
score.dtype
score1 = 100*score
print(score1)

score = predictions[30]
score.dtype
score1 = 100*score
print(score1)

score = predictions[31]
score.dtype
score1 = 100*score
print(score1)

score = predictions[32]
score.dtype
score1 = 100*score
print(score1)

score = predictions[33]
score.dtype
score1 = 100*score
print(score1)

score = predictions[34]
score.dtype
score1 = 100*score
print(score1)


score = predictions[35]
score.dtype
score1 = 100*score
print(score1)

score = predictions[36]
score.dtype
score1 = 100*score
print(score1)

score = predictions[37]
score.dtype
score1 = 100*score
print(score1)

score = predictions[38]
score.dtype
score1 = 100*score
print(score1)

score = predictions[39]
score.dtype
score1 = 100*score
print(score1)

score = predictions[40]
score.dtype
score1 = 100*score
print(score1)


score = predictions[41]
score.dtype
score1 = 100*score
print(score1)

score = predictions[42]
score.dtype
score1 = 100*score
print(score1)

score = predictions[43]
score.dtype
score1 = 100*score
print(score1)

score = predictions[44]
score.dtype
score1 = 100*score
print(score1)

score = predictions[45]
score.dtype
score1 = 100*score
print(score1)

score = predictions[46]
score.dtype
score1 = 100*score
print(score1)

score = predictions[47]
score.dtype
score1 = 100*score
print(score1)

score = predictions[48]
score.dtype
score1 = 100*score
print(score1)

score = predictions[49]
score.dtype
score1 = 100*score
print(score1)





1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
[33.012955]
[28.998783]
[51.701843]
[47.892307]
[44.69791]
[29.074839]
[41.032555]
[58.643703]
[30.730259]
[37.28396]
[46.993214]
[40.226044]
[25.26226]
[32.211678]
[32.758064]
[48.4278]
[42.903786]
[25.736103]
[39.688705]
[60.05689]


IndexError: index 20 is out of bounds for axis 0 with size 20